In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv('C:\DelayedFlights.csv', index_col=0)

In [3]:
# abans de reduir la dimensió de la mostra vull comprobar si hi ha elements 
# duplicats
len(df)

1936758

In [4]:
# elimino els elements duplicats, torno a demanar el tamany del dataframe i veig
# que hi ha dos registres menys, per tant sí que hi havia elements duplicats.

dfclean = df.drop_duplicates()
len(dfclean) == len(df)
len(dfclean)

1936756

### Exercici 1
#### Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un Pandas Dataframe. Explora les dades que conté, explica breument quines variables hi ha i queda’t únicament amb les columnes que consideris rellevants. Justifica la teva elecció. Redueix la dimensió del dataset de manera aleatòria per tal d'obtenir un dataset de només 200.000 registres. Tots els exercicis s'han de fer amb aquest dataset reduït.

Les variables que hi ha són: any, mes, dia del mes, dia de la setmana, hora de sortida, CRSDeptime és una declaració legal que permet la sortida de l'avió després de fer el manteniment. l'hora d'arribada,CRS ArrTime la certificació de l'hora d'arribada, flight num és el número de vol, tail num és el número de cua de l'avió, actual elapsed time és el temps que l'avió pasa a l'aire, crse lapsed time és l'estimació de temps de vol, air time és el temps des que l'avió abandona la superfície de recolçament fins que toca terra. Origin és d'es d'on surt l'avió, Destination és on aterra, Distance és la distància. Unique Carrier són les aerolínies. TaxiIn y TaxiOut són el temps que l'avió passa a terra abans d'enlairar-se o d'arribar al punt final.
 Cancelled, vol dir vol cancelat i el codi de cancelació. Diverted vol dir que l'avió no ha pogut arribar a la seva destinació final. Carrier Delay vol dir problemes de manteniment o circulació. WeatherDelay és el retard per motiu de temps. NAS delay vol dir una demora que podria corregir-se amb mesures correctives per part dels aeroports i de l'autoritat reguladora de l'aviació. Security delay vol dir la demora de seguretat o el reembarcament per violacions de seguretat.Late aircraft delay vol dir la demora a l'arribada a un aeroport perque ha arribat tard a un aeroport anterior.

In [5]:
# Redueixo el Dataset per obtenir 200000 registres de forma aleatòria
df2 = dfclean.sample(n=200000).copy()
len(df2)
# també trec informació completa de totes les columnes,
# la quantitat de valors nuls i no nuls i el tipus de variables.
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 3479625 to 1070079
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Year               200000 non-null  int64  
 1   Month              200000 non-null  int64  
 2   DayofMonth         200000 non-null  int64  
 3   DayOfWeek          200000 non-null  int64  
 4   DepTime            200000 non-null  float64
 5   CRSDepTime         200000 non-null  int64  
 6   ArrTime            199259 non-null  float64
 7   CRSArrTime         200000 non-null  int64  
 8   UniqueCarrier      200000 non-null  object 
 9   FlightNum          200000 non-null  int64  
 10  TailNum            199998 non-null  object 
 11  ActualElapsedTime  199121 non-null  float64
 12  CRSElapsedTime     199983 non-null  float64
 13  AirTime            199121 non-null  float64
 14  ArrDelay           199121 non-null  float64
 15  DepDelay           200000 non-null  float64


In [6]:
# he creat una nova columna per saber el temps total de vol, incloent el temps que l'avió passa a l'aira
# i el rodament de sortida i el rodament d'arribada.

df2.insert(4,'TotalTime',df2['AirTime']+df2['TaxiIn']+df2['TaxiOut'])

# per evitar que hi hagi molts decimals faig arrodoniment.

df2['TotalTime'] = df2['TotalTime'].round(2)

# he creat una nova columna amb la combinació d'origin i 
#dest per poder saber quines rutes són les més llargues o més curtes

df2.insert(5,'Rute',df2['Origin'].str.cat(df2['Dest'], sep='-'))

In [7]:
# canvio els dies de la setmana en número pel nom, crec que és més entenedor

days = {1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday',7:'Sunday'}
df2['DayOfWeek'] = df2['DayOfWeek'].map(days)

# canvio els dies dels mesos per els noms.

months = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
df2['Month'] = df2['Month'].map(months)

Tal com diuen a un dels tutorials una de las primeres columnes que es poden eliminar són les columnes amb més valors nuls, en aquest cas serien CarrierDelay, WeatherDelay, NASDelay, SecurityDelay,LateAircraftDelay. A més aquestes dades estarien duplicades amb les dades ArrDelay i DepDelay, també tenim dades duplicades a FlightNum i a TailNum, els dos serveixen per registrar el tipus d'avió, pero no cal tenir les dues. Per l'altre banda un cop he creat una nova columna amb el temps total, TaxiIn i Taxi Out deixen de ser importants, ara bé, no esborraré AirTime perque necessitaré aquesta informació per ver la velocitat mitjana de vol.

In [8]:
df2 = df2.drop(['CRSDepTime','CRSArrTime','CRSElapsedTime','ActualElapsedTime','TaxiIn','TaxiOut','FlightNum','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','CancellationCode'], axis =1)


In [9]:
df2.head()

,Year,Month,DayofMonth,DayOfWeek,TotalTime,Rute,DepTime,ArrTime,UniqueCarrier,TailNum,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,Cancelled,Diverted
3479625,2008,Jun,14,Saturday,123.0,ORD-LGA,745.0,1048.0,AA,N585AA,103.0,13.0,15.0,ORD,LGA,733,0,0
3973573,2008,Jul,6,Sunday,359.0,BWI-SEA,2009.0,2308.0,FL,N285AT,318.0,28.0,10.0,BWI,SEA,2335,0,0
687892,2008,Feb,27,Wednesday,112.0,TPA-IAD,1236.0,1428.0,WN,N399WN,99.0,68.0,81.0,TPA,IAD,810,0,0
5167223,2008,Sep,11,Thursday,130.0,ATL-DFW,1900.0,2010.0,FL,N915AT,103.0,-1.0,6.0,ATL,DFW,732,0,0
3517919,2008,Jun,20,Friday,61.0,MIA-TPA,1737.0,1838.0,AA,N3CAAA,43.0,178.0,177.0,MIA,TPA,204,0,0


### Exercici 2 
#### Fes un informe complet del dataset:

Resumeix estadísticament el dataset i les columnes d’interès. Fes una anàlisi estadístic del que consideris rellevant.
Troba quantes dades faltants hi ha per columna.
Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...).
Fes una taula de les aerolínies amb més endarreriments acumulats.
Quins són els vols més llargs? I els més endarrerits? Busca les rutes més llargues i les que acumulen més retards.
Aporta allò que consideris rellevant.

In [10]:
# per fer l'anàlisi he eliminat les columnes d'any, mes, hora d'arribada, hora de sortida, vols cancelats i vols que no arriben
# al destí. D'aquesta manera em puc centrar als endarreriments de sortida, d'arribada, la distància i el temps de vol i el temps
# total
df3 = df2.drop(['Year','DayofMonth','DepTime','ArrTime','Cancelled','Diverted'], axis=1)
df3.describe().round(2)

,TotalTime,AirTime,ArrDelay,DepDelay,Distance
count,199121.00,199121.00,199121.00,200000.00,200000.00
mean,133.37,108.38,42.25,43.26,766.70
std,72.10,68.70,56.58,53.25,574.91
min,18.00,1.00,-63.00,6.00,31.00
25%,80.00,58.00,9.00,12.00,338.00
50%,116.00,90.00,24.00,24.00,607.00
75%,165.00,137.00,56.00,54.00,998.00
max,667.00,644.00,1350.00,1358.00,4962.00


Si mirem els percentils veiem que en quant a duració de vol, el 25% es troba per sota dels 80 minuts, el 50% es troba per sota del 116 minuts i el 75% es troba per sota dels 165 minuts, i el màxim es troba als 667, per lo que es pot deduir que el tràfic es concentra més en tràfic proper vers tràfic internacional que tindria més duració de vol. Air time segueix també aquest patró pero els minuts del 25%,50% y 75%, amb 58 minuts, 90 minuts i 137 minuts respectivament. Això també estaria relacionat amb la Distància recorreguda. El 25% es troba per sota de 338 milles, uns 543 km, el 50% es troba per sota dels 607 milles, 976 km i el 75% es troba per sota de 998 milles, 1606.125 km, però la distància màxima d'una ruta són 4962 milles, uns 7985.5 km.

Si mirem els percentils amb els retrasos. En quant a arribades, el 25% dels avions es troba per sota dels 9 minuts, el 50% dels avions es troba per sota dels 24 minuts i el 75% dels avions es troba per sota dels 56 minuts, per tant la majoria d'endarreriments es troba dins d'un hora, per tant el màxim que veiem de 1350 minuts es pot considerar excepcional. En quant a les sortides, sabem que cap avió surt a l'hora, com a mínim hi ha un retràs de 6 minuts. Ara bé, un 25% dels avions es troba per sota dels 12 minuts, un 50% es troba per sota dels 24 minuts, i un 75% es troba per sota dels 53 minuts.

És curiós que la mitja de l'hora d'arribada i l'hora de sortida amb vols cancelats és quasi bé el mateix, d'uns 42 i 43 minuts. 

Considerable també és el màxim de temps d'endarreriment, 1350 minuts de l'arribada (unes 22.5 hores) i 1358 en el cas de les sortides (22.63 hores), en el cas de mínim cal veure que cap avió ha sortit a l'hora, com a mínim hi ha un retràs de 6 minuts, mentre que a les arribades hi ha un avió que ha arribat 63 minuts abans de l'hora convinguda.

In [11]:
# per saber les dades faltants per columna

df2.isnull().sum()

Year               0
Month              0
DayofMonth         0
DayOfWeek          0
TotalTime        879
Rute               0
DepTime            0
ArrTime          741
UniqueCarrier      0
TailNum            2
AirTime          879
ArrDelay         879
DepDelay           0
Origin             0
Dest               0
Distance           0
Cancelled          0
Diverted           0
dtype: int64

In [12]:
# velocitat mitjana = distancia / temps 
# no incloc el taxi in i taxi out perque l'avió està a terra i la seva velocidad és molt baixa.

df2.insert(5,'VelocitatMitjana',df2['Distance']/df2['AirTime'])
df2.head()

,Year,Month,DayofMonth,DayOfWeek,TotalTime,VelocitatMitjana,Rute,DepTime,ArrTime,UniqueCarrier,TailNum,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,Cancelled,Diverted
3479625,2008,Jun,14,Saturday,123.0,7.116505,ORD-LGA,745.0,1048.0,AA,N585AA,103.0,13.0,15.0,ORD,LGA,733,0,0
3973573,2008,Jul,6,Sunday,359.0,7.342767,BWI-SEA,2009.0,2308.0,FL,N285AT,318.0,28.0,10.0,BWI,SEA,2335,0,0
687892,2008,Feb,27,Wednesday,112.0,8.181818,TPA-IAD,1236.0,1428.0,WN,N399WN,99.0,68.0,81.0,TPA,IAD,810,0,0
5167223,2008,Sep,11,Thursday,130.0,7.106796,ATL-DFW,1900.0,2010.0,FL,N915AT,103.0,-1.0,6.0,ATL,DFW,732,0,0
3517919,2008,Jun,20,Friday,61.0,4.744186,MIA-TPA,1737.0,1838.0,AA,N3CAAA,43.0,178.0,177.0,MIA,TPA,204,0,0


In [13]:
# Creo una columna per saber si l'avió arriba amb endarreriment apareixerà True
# i si arriba a l'hora apareixerà false.
df2['Retraso'] =(df2['ArrDelay'] > 0) | (df2['DepDelay'] > 0)
df2.head()

,Year,Month,DayofMonth,DayOfWeek,TotalTime,VelocitatMitjana,Rute,DepTime,ArrTime,UniqueCarrier,TailNum,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,Cancelled,Diverted,Retraso
3479625,2008,Jun,14,Saturday,123.0,7.116505,ORD-LGA,745.0,1048.0,AA,N585AA,103.0,13.0,15.0,ORD,LGA,733,0,0,True
3973573,2008,Jul,6,Sunday,359.0,7.342767,BWI-SEA,2009.0,2308.0,FL,N285AT,318.0,28.0,10.0,BWI,SEA,2335,0,0,True
687892,2008,Feb,27,Wednesday,112.0,8.181818,TPA-IAD,1236.0,1428.0,WN,N399WN,99.0,68.0,81.0,TPA,IAD,810,0,0,True
5167223,2008,Sep,11,Thursday,130.0,7.106796,ATL-DFW,1900.0,2010.0,FL,N915AT,103.0,-1.0,6.0,ATL,DFW,732,0,0,True
3517919,2008,Jun,20,Friday,61.0,4.744186,MIA-TPA,1737.0,1838.0,AA,N3CAAA,43.0,178.0,177.0,MIA,TPA,204,0,0,True


In [14]:
# les 5 rutes amb més llargues:
print('les 5 rutes més llargues són: ')
df2.groupby(by = "Rute")["Distance"].max().sort_values(ascending=False).head(5)

les 5 rutes més llargues són: 


Rute
EWR-HNL    4962
HNL-EWR    4962
HNL-ATL    4502
ATL-HNL    4502
ORD-HNL    4243
Name: Distance, dtype: int64

In [15]:
# les 5 rutes amb menys curtes:
print('les 5 rutes més curtes són: ')
df2.groupby(by = "Rute")["Distance"].min().sort_values(ascending=True).head(5)

les 5 rutes més curtes són: 


Rute
PSG-WRG    31
WRG-PSG    31
SNA-LAX    36
LAX-SNA    36
MKG-GRR    41
Name: Distance, dtype: int64

In [16]:
# els 5 vols més endarrerits de sortida
print('els 5 vols més endarrerits de sortida són :')
df2.groupby(by = "TailNum")["DepDelay"].max().sort_values(ascending=False).head(5)

els 5 vols més endarrerits de sortida són :


TailNum
N501US    1358.0
N583NW    1348.0
N4WLAA    1300.0
N521US    1258.0
N593NW    1193.0
Name: DepDelay, dtype: float64

In [17]:
# els 5 vols més endarrerits d'arribada
print('els 5 vols més endarrerits darribada són:')
df2.groupby(by = "TailNum")["ArrDelay"].max().sort_values(ascending=False).head(5)

els 5 vols més endarrerits darribada són:


TailNum
N583NW    1350.0
N501US    1344.0
N4WLAA    1286.0
N521US    1263.0
N593NW    1189.0
Name: ArrDelay, dtype: float64

In [18]:
# les 5 aerolínies amb més minuts demores de sortida
print('les aerolínies amb més minuts en demores de sortida són:')
df2.groupby(by ="UniqueCarrier")["ArrDelay"].sum().sort_values(ascending=False).head(5)

les aerolínies amb més minuts en demores de sortida són:


UniqueCarrier
WN    1161269.0
AA     925636.0
UA     701549.0
MQ     672977.0
OO     611040.0
Name: ArrDelay, dtype: float64

In [19]:
# les 5 aerolínies amb més minuts en demores d'arribada
print('les aerolínies amb més minuts en demores darribada són:')
df2.groupby(by="UniqueCarrier")["DepDelay"].sum().sort_values(ascending=False).head(5)

les aerolínies amb més minuts en demores darribada són:


UniqueCarrier
WN    1346215.0
AA     923917.0
UA     738540.0
MQ     655111.0
OO     606384.0
Name: DepDelay, dtype: float64

In [20]:
# les 5 aerolínies que cancelen més vols
print('les aerolínies que cancelen més vols són:')
df2.groupby(by="UniqueCarrier")['Cancelled'].sum().sort_values(ascending=False).head(5)


les aerolínies que cancelen més vols són:


UniqueCarrier
OO    13
MQ    13
UA     9
9E     6
XE     6
Name: Cancelled, dtype: int64

In [21]:
# les 5 aerolínies que cancelen menys vols
print('les aerolínies que cancelen menys vols són:')
df2.groupby(by="UniqueCarrier")['Cancelled'].sum().sort_values(ascending=True).head(5)

les aerolínies que cancelen menys vols són:


UniqueCarrier
FL    0
AQ    0
AS    0
B6    0
OH    0
Name: Cancelled, dtype: int64

In [23]:
# les 5 rutes amb més endarreriments:
print('les aerolínies amb més endarreriments són:')
df2.groupby(by="Rute")['Retraso'].sum().sort_values(ascending=False).head(5)

les aerolínies amb més endarreriments són:


Rute
LAX-SFO    486
ORD-LGA    467
SFO-LAX    420
LAS-LAX    375
ORD-EWR    375
Name: Retraso, dtype: int64

In [24]:
# les 5 rutes amb menys endarriments:
print('les aerolínies amb menys endarreriments són:')
df2.groupby(by="Rute")['Retraso'].sum().sort_values(ascending=True).head(5)

les aerolínies amb menys endarreriments són:


Rute
LAX-SNA    1
JAX-PIR    1
MSP-PWM    1
FSD-PIA    1
CVG-CAK    1
Name: Retraso, dtype: int64

### Exercici 3
#### Exporta el dataset net i amb les noves columnes a Excel

In [26]:
df2.to_excel("DelayedFlights.xlsx",
            sheet_name = 'Dades')